This notebook demonstrates projection of point cloud at the viewpoint of nearby panorma as rasters, and export clipped rasters.

In [ ]:
import numpy as np
# from matplotlib import cm
from matplotlib import pyplot as plt
# from pyproj import Transformer
# from scipy.spatial.transform import Rotation as R
from lidar.point_cloud_processings import project_las_to_equirectangular, fill_small_nans, resize_preserve_nans
import geopandas as gpd
import glob
import pyproj
import os
from skimage.io import imsave

### Input and outputs

In [ ]:
building_points_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping.geojson'
las_files_folder = r"/mnt/floorheightvolume/lidar_Wagga/clipped/"
out_folder=r"/mnt/floorheightvolume/lidar_Wagga/clipped_projected/"
os.makedirs(out_folder, exist_ok=True)

### Load building points

In [ ]:
gdf_building_points=gpd.read_file(building_points_file)
gdf_building_points=gdf_building_points[gdf_building_points["USAGE"]=="Residential"].reset_index(drop=True)
gdf_building_points.head()

## Test workflow with one building example

### Identify corresponding las file

In [ ]:
# building_ufi=1271
# gdf_building_points[gdf_building_points['UFI']==building_ufi]

In [ ]:
i=90
building_ufi=gdf_building_points.iloc[i]['UFI']
house_loc_left, house_loc_right = int(gdf_building_points.iloc[i]['house_loc_left']), int(gdf_building_points.iloc[i]['house_loc_right'])
las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]
las_file_path

### Reproject trajectory coordinates to match lidar points

In [ ]:
transformer = pyproj.Transformer.from_crs("EPSG:7844",  # GDA2020 geographic (lat/lon)
                                          "EPSG:7855",      # MGA Zone 55 (EPSG:7855)
                                          always_xy=True)
lat, lon, elev = [gdf_building_points.iloc[i]['LATITUDE'],gdf_building_points.iloc[i]['LONGITUDE'],gdf_building_points.iloc[i]['LTP_z_m']] # lidar data is in AHD
x_proj, y_proj = transformer.transform(lon, lat)  # lon, lat order
camera_pos_proj = [x_proj, y_proj, elev]
camera_angles=[gdf_building_points.iloc[i]['Heading_deg'], gdf_building_points.iloc[i]['Pitch_deg'], (-1)*gdf_building_points.iloc[i]['Roll_deg']] # TODO: figure out why reversed sign works

In [ ]:
gdf_building_points.iloc[i]['Pitch_deg']

In [ ]:
gdf_building_points.iloc[i]['LTP_z_m']

### Densify points (optional depending on performance)

In [ ]:
# out_densified_path=las_file_path.split('.')[0]+'_densified.las'
# pipeline_json = {
#     "pipeline": [
#         las_file_path,
#         {
#             "type": "filters.poisson",
#             "depth": 10, # Start with a mid-range depth (this controls resolution)
#         },
#         out_densified_path
#     ]
# }
# pipeline = pdal.Pipeline(json.dumps(pipeline_json))
# pipeline.execute()

### Project the point cloud as rasters
* Note: using the same width/height ratio as panoramas
* reduced resolution to improve sampling of surface points compared to background points

In [ ]:
upper_crop=0.25
lower_crop=0.6 # consistent with panorama clipping
width_panorama=11000
height_panorama=5500  # panoramas parameters
downscale_factor=4 # scale factor between panorama and projected lidar rasters
width=int(width_panorama/downscale_factor)
height=int(height_panorama/downscale_factor)
rgb, z, depth, classification, intensity = project_las_to_equirectangular(input_las=las_file_path, camera_pos=camera_pos_proj,
                                                               camera_angles=camera_angles, width=width, height=height)

### Interpolate gaps on elevation and intensity rasters

In [ ]:
# fill small holes
# z_arr_clipped_filled = fill_small_nans(z_arr_clipped, max_hole_size=5)
z_arr_filled = fill_small_nans(z, max_hole_size=10, nodata_value=9999)
intensity_filled = fill_small_nans(intensity, max_hole_size=10, nodata_value=255)

### Upsample to the same resolution as panorama

In [ ]:
# # resize while preserving remaining NaNs
# new_width = z_arr_clipped_filled.shape[0]*downscale_factor
# new_height = z_arr_clipped_filled.shape[1]*downscale_factor
# z_arr_filled_resampled=resize_preserve_nans(z_arr_clipped_filled,new_width,new_height)

z_filled_resampled=resize_preserve_nans(z_arr_filled,height_panorama, width_panorama, nodata_value=9999)
intensity_filled_resampled=resize_preserve_nans(intensity_filled, height_panorama, width_panorama, nodata_value=255)

### Clip projected rasters to the same region as panoramas
* only elevation and intensity rasters were upsampled for use in next steps

In [ ]:
z_processed=z_filled_resampled[int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama)),
                               house_loc_left:house_loc_right]
intensity_processed=intensity_filled_resampled[int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama)),
                                               house_loc_left:house_loc_right]

* other rasters are unprocessed and low resolution for visual check only

In [ ]:
min_row, max_row=int(round(upper_crop*height)), int(round(lower_crop*height))
min_col, max_col=int(round(house_loc_left/downscale_factor)), int(round(house_loc_right/downscale_factor))

rgb_arr_clipped=rgb[min_row:max_row, min_col:max_col,:]
class_arr_clipped=classification[min_row:max_row, min_col:max_col]
depth_arr_clipped=depth[min_row:max_row, min_col:max_col]

### Save rasters

In [ ]:
out_path_rgb=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_rgb.tif'))
out_path_elevation=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_elevation_resampled.tif'))
out_path_intensity=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_intensity_resampled.tif'))
out_path_classification=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_classification.tif'))
out_path_depth=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_depth.tif'))

In [ ]:
imsave(out_path_rgb, rgb_arr_clipped)
imsave(out_path_classification, class_arr_clipped)
imsave(out_path_depth, depth_arr_clipped)
imsave(out_path_elevation, z_processed)
imsave(out_path_intensity, intensity_processed)

## Batch apply to all buildings with nearby panoramas

In [ ]:
for i in range(len(gdf_building_points)):
    building_ufi=gdf_building_points.iloc[i]['UFI']
    house_loc_left, house_loc_right = int(gdf_building_points.iloc[i]['house_loc_left']), int(gdf_building_points.iloc[i]['house_loc_right'])
    try:
        # find las by UFI
        las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]
        
        out_path_rgb=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_rgb.tif'))
        out_path_elevation=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_elevation_resampled.tif'))
        out_path_intensity=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_intensity_resampled.tif'))
        out_path_classification=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_classification.tif'))
        out_path_depth=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_depth.tif'))

        if os.path.exists(out_path_rgb) and os.path.exists(out_path_elevation) and os.path.exists(out_path_depth)\
            and os.path.exists(out_path_intensity) and os.path.exists(out_path_classification):
            print('rasters exist, skipping...')
        else:
            # get viewpoint metadata
            lat, lon, elev = [gdf_building_points.iloc[i]['LATITUDE'],gdf_building_points.iloc[i]['LONGITUDE'],gdf_building_points.iloc[i]['LTP_z_m']] # lidar data is in AHD
            x_proj, y_proj = transformer.transform(lon, lat)  # lon, lat order
            camera_pos_proj = [x_proj, y_proj, elev]
            camera_angles=[gdf_building_points.iloc[i]['Heading_deg'], gdf_building_points.iloc[i]['Pitch_deg'], (-1)*gdf_building_points.iloc[i]['Roll_deg']]
            
            # project as rasters
            rgb, z, depth, classification, intensity = project_las_to_equirectangular(input_las=las_file_path, camera_pos=camera_pos_proj,
                                                                        camera_angles=camera_angles, width=width, height=height)
            
            # fill small holes
            z_arr_filled = fill_small_nans(z, max_hole_size=10, nodata_value=9999)
            intensity_filled = fill_small_nans(intensity, max_hole_size=10, nodata_value=255)

            # upsample
            z_filled_resampled=resize_preserve_nans(z_arr_filled,height_panorama, width_panorama, nodata_value=9999)
            intensity_filled_resampled=resize_preserve_nans(intensity_filled, height_panorama, width_panorama, nodata_value=255)

            # crop
            z_processed=z_filled_resampled[int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama)),
                                           house_loc_left:house_loc_right]
            intensity_processed=intensity_filled_resampled[int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama)),
                                                           house_loc_left:house_loc_right]
            
            min_row, max_row=int(round(upper_crop*height)), int(round(lower_crop*height))
            min_col, max_col=int(round(house_loc_left/downscale_factor)), int(round(house_loc_right/downscale_factor))
            rgb_arr_clipped=rgb[min_row:max_row, min_col:max_col,:]
            class_arr_clipped=classification[min_row:max_row, min_col:max_col]
            depth_arr_clipped=depth[min_row:max_row, min_col:max_col]

            #save
            imsave(out_path_rgb, rgb_arr_clipped)
            imsave(out_path_classification, class_arr_clipped)
            imsave(out_path_depth, depth_arr_clipped)
            imsave(out_path_elevation, z_processed)
            imsave(out_path_intensity, intensity_processed)

    except Exception as e:
        print(e)